In [2]:
import backtrader as bt
import pandas as pd
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import coint
from datetime import datetime

In [52]:
class PairTradingStrategy(bt.Strategy):
    params = dict(
        lookback=100,
        zscore_high=2,
        zscore_low=-2
    )

    def __init__(self):
        self.data0 = self.datas[0]
        self.data1 = self.datas[1]

        self.zscore = None
        self.order = None

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} {txt}")

    def next(self):
        if self.order:
            return
        
        # Check if there is enough data
        if len(self.data0) < self.params.lookback or len(self.data1) < self.params.lookback:
            return

        # Compute z-score
        prices0 = np.array(self.data0.get(size=self.params.lookback))
        prices1 = np.array(self.data1.get(size=self.params.lookback))
        spread = prices0 - prices1
        zscore = (spread[-1] - spread.mean()) / spread.std()

        fund = 1000
        size0 = fund / self.data0.get(size=1)[0]
        size1 = fund / self.data1.get(size=1)[0]
        if zscore > self.params.zscore_high:
            # Sell the spread (long CFG, short KEY)
            self.sell(data=self.data0, size=size0)
            self.buy(data=self.data1, size=size1)
            self.zscore = zscore
        elif zscore < self.params.zscore_low:
            # Buy the spread (long KEY, short CFG)
            self.sell(data=self.data1, size=size1)
            self.buy(data=self.data0, size=size0)
            self.zscore = zscore

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        # if order.status in [order.Completed]:
        #     if order.isbuy():
        #         self.log(f"BUY {order.data._name}: {order.executed.price}")
        #     else:
        #         self.log(f"SELL {order.data._name}: {order.executed.price}")

        self.order = None

In [53]:
def backtest_pair_trading(symbol1, symbol2, start_date, end_date):
    cerebro = bt.Cerebro()

    # Download data
    data1 = yf.download(symbol1, start=start_date, end=end_date)
    data2 = yf.download(symbol2, start=start_date, end=end_date)

    # Add the data to Cerebro
    cerebro.adddata(bt.feeds.PandasData(dataname=data1), name=symbol1)
    cerebro.adddata(bt.feeds.PandasData(dataname=data2), name=symbol2)

    # Add the strategy
    cerebro.addstrategy(PairTradingStrategy)

    # Set the starting cash
    cerebro.broker.setcash(10000.0)
        # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio", riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Days)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    # Run the backtest
    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
    results = cerebro.run()
    print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
       # Extract and print the results
    trade_analyzer = results[0].analyzers.trade_analyzer.get_analysis()
    sharpe_ratio = results[0].analyzers.sharpe_ratio.get_analysis()["sharperatio"]

    total_trades = trade_analyzer.total.closed
    winning_trades = trade_analyzer.won.total
    win_rate = winning_trades / total_trades * 100

    print(f"Total trades: {total_trades}")
    print(f"Winning trades: {winning_trades}")
    print(f"Trade win rate: {win_rate:.2f}%")
    print(f"Sharpe ratio: {sharpe_ratio:.2f}")

    max_drawdown = results[0].analyzers.drawdown.get_analysis()["max"]["drawdown"]

    print(f"Maximum drawdown: {max_drawdown:.2f}%")


In [54]:

start_date = '2010-01-01'
end_date = '2023-04-01'
symbol1 = 'CFG'
symbol2 = 'KEY'

backtest_pair_trading(symbol1, symbol2, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Starting Portfolio Value: 10000.00
Final Portfolio Value: 13904.96
Total trades: 6
Winning trades: 3
Trade win rate: 50.00%
Sharpe ratio: 0.25
Maximum drawdown: 69.26%
